In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error , r2_score
import joblib
%matplotlib inline

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/logan0501/Datasets/main/T1.csv')
data.rename(columns = {'LV ActivePower (kW)':'ActivePower(kW)',
                       "Wind Speed (m/s)":"WindSpeed(m/s)",
                       "Wind Direction (°)":"WindDirection","Theoretical_Power_Curve (KWh)":"TheoreticalPowerCurve(KWh)"},
            inplace = True)
data.head()

,Date/Time,ActivePower(kW),WindSpeed(m/s),TheoreticalPowerCurve(KWh),WindDirection
0,01 01 2018 00:00,380.047791,5.311336,416.328908,259.994904
1,01 01 2018 00:10,453.769196,5.672167,519.917511,268.641113
2,01 01 2018 00:20,306.376587,5.216037,390.900016,272.564789
3,01 01 2018 00:30,419.645905,5.659674,516.127569,271.258087
4,01 01 2018 00:40,380.650696,5.577941,491.702972,265.674286


In [ ]:
data.shape

(50530, 5)

In [ ]:
data.describe()

,ActivePower(kW),WindSpeed(m/s),TheoreticalPowerCurve(KWh),WindDirection
count,50530.000000,50530.000000,50530.000000,50530.000000
mean,1307.684332,7.557952,1492.175463,123.687559
std,1312.459242,4.227166,1368.018238,93.443736
min,-2.471405,0.000000,0.000000,0.000000
25%,50.677890,4.201395,161.328167,49.315437
50%,825.838074,7.104594,1063.776283,73.712978
75%,2482.507568,10.300020,2964.972462,201.696720
max,3618.732910,25.206011,3600.000000,359.997589


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50530 entries, 0 to 50529
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Date/Time                   50530 non-null  object 
 1   ActivePower(kW)             50530 non-null  float64
 2   WindSpeed(m/s)              50530 non-null  float64
 3   TheoreticalPowerCurve(KWh)  50530 non-null  float64
 4   WindDirection               50530 non-null  float64
dtypes: float64(4), object(1)
memory usage: 1.9+ MB


In [ ]:
data.isnull().any()

Date/Time                     False
ActivePower(kW)               False
WindSpeed(m/s)                False
TheoreticalPowerCurve(KWh)    False
WindDirection                 False
dtype: bool

## Data Preprocessing

In [ ]:
data['Date/Time'] = pd.to_datetime(data['Date/Time'],format='%d %m %Y %H:%M')
data['year'] = data['Date/Time'].dt.year 
data['month'] = data['Date/Time'].dt.month 
data['day'] = data['Date/Time'].dt.day
data['Hour'] = data['Date/Time'].dt.hour 
data['minute'] = data['Date/Time'].dt.minute 
data.head()

,Date/Time,ActivePower(kW),WindSpeed(m/s),TheoreticalPowerCurve(KWh),WindDirection,year,month,day,Hour,minute
0,2018-01-01 00:00:00,380.047791,5.311336,416.328908,259.994904,2018,1,1,0,0
1,2018-01-01 00:10:00,453.769196,5.672167,519.917511,268.641113,2018,1,1,0,10
2,2018-01-01 00:20:00,306.376587,5.216037,390.900016,272.564789,2018,1,1,0,20
3,2018-01-01 00:30:00,419.645905,5.659674,516.127569,271.258087,2018,1,1,0,30
4,2018-01-01 00:40:00,380.650696,5.577941,491.702972,265.674286,2018,1,1,0,40


In [ ]:
data["Date/Time"] = pd.to_datetime(data["Date/Time"], format = "%d %m %Y %H:%M", errors = "coerce")
data

,Date/Time,ActivePower(kW),WindSpeed(m/s),TheoreticalPowerCurve(KWh),WindDirection,year,month,day,Hour,minute
0,2018-01-01 00:00:00,380.047791,5.311336,416.328908,259.994904,2018,1,1,0,0
1,2018-01-01 00:10:00,453.769196,5.672167,519.917511,268.641113,2018,1,1,0,10
2,2018-01-01 00:20:00,306.376587,5.216037,390.900016,272.564789,2018,1,1,0,20
3,2018-01-01 00:30:00,419.645905,5.659674,516.127569,271.258087,2018,1,1,0,30
4,2018-01-01 00:40:00,380.650696,5.577941,491.702972,265.674286,2018,1,1,0,40
...,...,...,...,...,...,...,...,...,...,...
50525,2018-12-31 23:10:00,2963.980957,11.404030,3397.190793,80.502724,2018,12,31,23,10
50526,2018-12-31 23:20:00,1684.353027,7.332648,1173.055771,84.062599,2018,12,31,23,20
50527,2018-12-31 23:30:00,2201.106934,8.435358,1788.284755,84.742500,2018,12,31,23,30
50528,2018-12-31 23:40:00,2515.694092,9.421366,2418.382503,84.297913,2018,12,31,23,40


## Splitting the dataset

In [ ]:
X=data[['WindSpeed(m/s)','WindDirection']]
X.head()

,WindSpeed(m/s),WindDirection
0,5.311336,259.994904
1,5.672167,268.641113
2,5.216037,272.564789
3,5.659674,271.258087
4,5.577941,265.674286


In [ ]:
y = data['ActivePower(kW)']
y.head()

0    380.047791
1    453.769196
2    306.376587
3    419.645905
4    380.650696
Name: ActivePower(kW), dtype: float64

In [ ]:
X_train, X_test,y_train, y_test = train_test_split(X,y ,
                                   random_state=6, 
                                   test_size=0.25)

## Importing the regression Models

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score,r2_score,mean_squared_error
xgr=XGBRegressor()
rf=RandomForestRegressor()
lr=LinearRegression()
dt=DecisionTreeRegressor()
sm=SVR()

## Fitting the models with the dataset

In [ ]:
model_xg=xgr.fit(X_train,y_train)
y_xg=model_xg.predict(X_test)
# model_rf=rf.fit(X_train,y_train)
# y_rf=model_rf.predict(X_test)
# model_lr=lr.fit(X_train,y_train)
# y_lr=model_lr.predict(X_test)
# model_dt=dt.fit(X_train,y_train)
# y_dt=model_dt.predict(X_test)
# model_sm=sm.fit(X_train,y_train)
# y_sm=model_sm.predict(X_test)

## Checking the metrics

In [ ]:
print('R2-xgb',r2_score(y_test,y_xg))
print('RMSE-xgb',np.sqrt(mean_squared_error(y_test,y_xg)))

# print('R2-rf',r2_score(y_test,y_rf))
# print('RMSE-rf',np.sqrt(mean_squared_error(y_test,y_rf)))

# print('R2-lr',r2_score(y_test,y_lr))
# print('RMSE-lr',np.sqrt(mean_squared_error(y_test,y_lr)))

# print('R2-dt',r2_score(y_test,y_dt))
# print('RMSE-dt',np.sqrt(mean_squared_error(y_test,y_dt)))

# print('R2-svm',r2_score(y_test,y_sm))
# print('RMSE-svm',np.sqrt(mean_squared_error(y_test,y_sm)))

R2-xgb 0.9197743106205652
RMSE-xgb 370.6768884049128


In [ ]:
# import pickle
# file_name = "xgb_reg.pkl"

# # save
# pickle.dump(xgb_model, open(file_name, "wb"))

In [ ]:
# model_xg.save_model('test_model.bin')

In [ ]:
# data=[[5.311336,259.994904]]  
# df = pd.DataFrame(data, columns=[	'WindSpeed(m/s)','WindDirection'])
# xgr.predict(df)

## IBM Deployment

In [ ]:
!pip install -U ibm-watson-machine-learning

     |████████████████████████████████| 1.8 MB 13.7 MB/s eta 0:00:01
  Attempting uninstall: ibm-watson-machine-learning
    Found existing installation: ibm-watson-machine-learning 1.0.255
    Uninstalling ibm-watson-machine-learning-1.0.255:
      Successfully uninstalled ibm-watson-machine-learning-1.0.255


In [ ]:
from ibm_watson_machine_learning import APIClient
import json

## Authenticate and set Space

t1xJwH_pNvesyStso2tawTlpypHX0HEQJVMev99cmAtK

In [ ]:
wml_credentials = {
    "apikey":"t1xJwH_pNvesyStso2tawTlpypHX0HEQJVMev99cmAtK",
    "url":"https://us-south.ml.cloud.ibm.com"
}

In [ ]:
wml_client = APIClient(wml_credentials)

In [ ]:
wml_client.spaces.list()
#9ebdfdcd-9254-4c80-bfaf-df2ab971a807

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  -----  ------------------------
ID                                    NAME   CREATED
9ebdfdcd-9254-4c80-bfaf-df2ab971a807  XGB_1  2022-10-24T16:22:07.291Z
------------------------------------  -----  ------------------------


In [ ]:
SPACE_ID= "9ebdfdcd-9254-4c80-bfaf-df2ab971a807"

In [ ]:
wml_client.set.default_space(SPACE_ID)

'SUCCESS'

In [ ]:
wml_client.software_specifications.list(100)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1

In [ ]:
import sklearn
sklearn.__version__

'1.0.2'

In [ ]:
MODEL_NAME = 'XGB_1'
DEPLOYMENT_NAME = 'XGB_1'
DEMO_MODEL = model_xg

In [ ]:
# Set Python Version
software_spec_uid = wml_client.software_specifications.get_id_by_name('runtime-22.1-py3.9')

In [ ]:
# Setup model meta
model_props = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME, 
    wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0', 
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid 
}

In [ ]:
#Save model

model_details = wml_client.repository.store_model(
    model=DEMO_MODEL, 
    meta_props=model_props, 
    training_data=X_train, 
    training_target=y_train
)

In [ ]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'label_column': 'ActivePower(kW)',
  'schemas': {'input': [{'fields': [{'name': 'WindSpeed(m/s)',
       'type': 'float64'},
      {'name': 'WindDirection', 'type': 'float64'}],
     'id': '1',
     'type': 'struct'}],
   'output': []},
  'software_spec': {'id': '12b83a17-24d8-5082-900f-0ab31fbfd3cb',
   'name': 'runtime-22.1-py3.9'},
  'type': 'scikit-learn_1.0'},
 'metadata': {'created_at': '2022-10-24T16:27:10.568Z',
  'id': '4e6c5b96-fab5-44db-ac39-3f744f3cb469',
  'modified_at': '2022-10-24T16:27:14.343Z',
  'name': 'XGB_1',
  'owner': 'IBMid-666002LP2L',
  'resource_key': '8f2e5b48-b33a-4255-89d9-33adb5729b2d',
  'space_id': '9ebdfdcd-9254-4c80-bfaf-df2ab971a807'},
 'system': {'warnings': []}}

In [ ]:
model_id = wml_client.repository.get_model_id(model_details)
model_id

'4e6c5b96-fab5-44db-ac39-3f744f3cb469'

In [ ]:
# Set meta
deployment_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME:DEPLOYMENT_NAME, 
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [ ]:
# Deploy
deployment = wml_client.deployments.create(
    artifact_uid=model_id, 
    meta_props=deployment_props 
)



#######################################################################################

Synchronous deployment creation for uid: '4e6c5b96-fab5-44db-ac39-3f744f3cb469' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='0644c680-478f-475f-bc23-2a64fc6490a5'
------------------------------------------------------------------------------------------------


